In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dropout, Dense,
                                    SimpleRNN, Reshape, BatchNormalization, Input,
                                     )
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import matplotlib.pyplot as plt


In [6]:
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
data_train_path = 'Dataset/Train'
data_val_path = 'Dataset/Validation'
data_test_path = 'Dataset/Test'

In [8]:
img_width = 224
img_height = 224
batch_size = 32

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    validation_split=0.5,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary'
)

# Load validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_val_path,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    label_mode='binary'
)

# Load test data
test_ds = tf.keras.utils.image_dataset_from_directory(
    data_test_path,
    image_size=(img_height, img_width),
    shuffle=False,
    batch_size=batch_size,
    label_mode='binary'
)

Found 140002 files belonging to 2 classes.
Using 70001 files for training.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [14]:
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [15]:
def build_improved_model(input_shape=(img_height, img_width, 3)):
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    
    # Base model (MobileNetV2)
    base_model = MobileNetV2(
        weights='imagenet', 
        include_top=False, 
        input_shape=input_shape
    )
    base_model.trainable = False
    
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, -1))(x)
    
    # Improved RNN layers
    x = SimpleRNN(128, return_sequences=True)(x)
    x = Dropout(0.4)(x)
    x = SimpleRNN(64)(x)
    x = Dropout(0.4)(x)
    
    # Enhanced classifier
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs, outputs)

In [16]:
model = build_improved_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

In [10]:
callbacks = [
    ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, mode='max'),
    ModelCheckpoint(
        filepath='best_model.keras',
        save_best_only=True,
        monitor='val_auc',
        mode='max'
    )
]

In [11]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks
)

Epoch 1/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1809s 820ms/step - accuracy: 0.6107 - auc: 0.6533 - loss: 0.8280 - val_accuracy: 0.7486 - val_auc: 0.8370 - val_loss: 0.5165 - learning_rate: 1.0000e-04
Epoch 2/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1933s 883ms/step - accuracy: 0.7075 - auc: 0.7775 - loss: 0.6020 - val_accuracy: 0.7593 - val_auc: 0.8506 - val_loss: 0.4990 - learning_rate: 1.0000e-04
Epoch 3/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1330s 608ms/step - accuracy: 0.7401 - auc: 0.8147 - loss: 0.5337 - val_accuracy: 0.7569 - val_auc: 0.8560 - val_loss: 0.5063 - learning_rate: 1.0000e-04
Epoch 4/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1344s 614ms/step - accuracy: 0.7606 - auc: 0.8373 - loss: 0.4981 - val_accuracy: 0.7585 - val_auc: 0.8620 - val_loss: 0.5040 - learning_rate: 1.0000e-04
Epoch 5/50
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 1544s 679ms/step - accuracy: 0.7647 - auc: 0.8448 - loss: 0.4850 - val_accuracy: 0.7574 - val_auc: 0.8651 - val_loss: 0.4852 - learning_rate: 1.0000e-04
Epoch 6/50
2188/2188 ━━━━

In [1]:
def plot_history(history):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('CNN+LSTM Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('CNN+LSTM Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_history(history)

In [25]:
model.save('model2.keras')